In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
from filtering import between
from datetime import date, datetime

folder = Path("base")

In [5]:
from preprocess_utils import load_jsonb, save_jsonb 
from tqdm import tqdm
from urllib.parse import urlparse
for art in load_jsonb("final.jsonb"):
    url = art["url"]
    parsed = urlparse(url).netloc.split(".")
    server = parsed[-1]
    if server == "cz":
        server = parsed[-2]

    save_jsonb([art], folder / f"{server}.jsonb", "a", show_progress=False)



    



100%|██████████| 3193720/3193720 [14:31<00:00, 3664.86it/s]


In [6]:
# Add from cz:403 to aktualne becauose of the same domain

for art in load_jsonb(folder / "cz:443.jsonb"):
    save_jsonb([art], folder / "aktualne.jsonb", "a", show_progress=False)


100%|██████████| 1/1 [00:00<00:00, 1490.51it/s]


In [7]:
from preprocess_utils import articles_num
print(list(articles_num(folder)))

[('ihned.jsonb', 31643), ('denik.jsonb', 1636639), ('seznamzpravy.jsonb', 77442), ('irozhlas.jsonb', 201681), ('idnes.jsonb', 530855), ('aktualne.jsonb', 165799), ('novinky.jsonb', 549661)]


In [10]:
# Custom config for each server
configs = {
    "idnes": {},
    "denik": {},
    "aktualne": {},
    "irozhlas": {},
    "seznamzpravy": {},
    "novinky": {},
}

default = {
    "article_length": between(300, None),
    "avg_word_length": between(4.0, None),
    "word_num_ratio": between(None, 0.22),
    "headline_length": between(16, None),
    "brief_length": between(40, None),
    "non-alpha_ratio": between(None, 0.045),
    "publication_date": between(date(2000, 1, 1), date(2022, 8, 31)),
}

In [ ]:
from filtering import create_filter_by_stats, create_filter_by_cz_lang, create_config
from preprocess_utils import load_jsonb, save_jsonb
from postprocessing_utils import postprocess_authors, filter_author, add_server, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date, as_Article

ratio = 0.75
for f in folder.iterdir():
    filters = [
        create_filter_by_cz_lang(),
        create_filter_by_stats(create_config(configs[f.stem])), 
        ]

    augmentations = [add_server, filter_author, postprocess_authors, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date, as_Article]

    articles = load_jsonb(f)
    aritcles = (article for article in articles if all(f(article) for f in filters))

    articles = (augmentation(article) for article in articles for augmentation in augmentations)
    save_jsonb(articles, f"article_json_filtered/{f.stem}.jsonl")

         



    